In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import scipy
from numpy.random import default_rng
import scipy.linalg as sla
import scipy.linalg.interpolative as sli

from frequentDirections import FrequentDirections
import time
import h5py
from rSVDsp import *

import matplotlib.pyplot as plt

In [90]:
def streaming_ridge_leverage(A, k, t, epsilon, delta, c, rng=default_rng()):
    """
    from http://arxiv.org/abs/1511.07263
    A should be n by d
    k is the rank of the projection with theoretical guarantees.
    t is the stored column size
    epsilon: accuracy parameter
    delta: (1 - delta) is the success probability
    c: oversampling parameter
    choose epsilon and delta to be less than one.
    """
    n, d = np.shape(A)
    count = 0
    C = np.zeros((n, t))  # Stores actual column subset
    D = np.zeros((n, t))  # Stores a queue of new columns
    frobA = 0

    tau_old = np.ones((t))  # Initialize sampling probabilities
    tau = tau_old
    l = 2  # parameter for FrequentDirections

    sketcher = FrequentDirections(n, (l + 1) * k)
    for i in range(d):
        a = A[:, i].T
        sketcher.append(a)
        B = sketcher.get().T
        # print(np.shape(B))
        if count < t:
            D[:, count] = A[:, i]
            frobA = frobA + sla.norm(a) ** 2
            count = count + 1
        else:
            tau = np.minimum(tau_old, ApproximateRidgeScores(B, C, frobA, k))
            tau_D = ApproximateRidgeScores(B, D, frobA, k)
            # print(tau)
            # print(tau_D)
            for j in range(t):
                if not np.all(C[:, j] == 0):
                    prob_rej = 1.0 - tau[j] / tau_old[j]
                    roll = rng.random()
                    if roll < prob_rej:
                        C[:, j] = 0
                        tau_old[j] = 1.0
                    else:
                        tau_old[j] = tau[j]
                if np.all(C[:, j] == 0):
                    for l in range(t):
                        prob = (
                            tau_D[l]
                            * c
                            * (k * np.log(k) + k * np.log(1.0 / delta) / epsilon)
                            / t
                        )
                        roll = rng.random()
                        if roll < prob:
                            C[:, j] = D[:, l]
                            tau_old[j] = tau_D[l]
            count = 0

    return C


def rID_streaming_ridge_leverage_old(A, k, t, epsilon, delta, c, os, rng=default_rng()):
    """
    from http://arxiv.org/abs/1511.07263
    A should be n by d
    k is the rank of the projection with theoretical guarantees.
    t is the stored column size
    epsilon: accuracy parameter
    delta: (1-delta) is the success probability
    c: oversampling parameter
    os: oversampling number for random projection
    choose epsilon and delta to be less than one.
    """
    n, d = np.shape(A)
    ll = k + os
    Omg = rng.standard_normal(size=(ll, n))

    # t = np.ceil(32. * c * (k*np.log(k)+k*np.log(1./delta)/epsilon)).astype(int)
    # print(t)

    count = 0
    nBlock = 0
    C = np.zeros((n, t))  # Stores actual column subset
    D = np.zeros((n, t))  # Stores a queue of new columns
    frobA = 0

    Q = np.zeros((t, d))
    C_old = C
    OmgA = np.zeros((ll, d))

    tau_old = np.ones((t))  # Initialize sampling probabilities
    tau = tau_old
    l = 2  # parameter for FrequentDirections

    sketcher = FrequentDirections(n, (l + 1) * k)
    for i in range(d):
        a = A[:, i].T
        sketcher.append(a)
        B = sketcher.get().T
        OmgA[:, i] = Omg @ A[:, i]
        # print(np.shape(B))
        if count < t:
            D[:, count] = A[:, i]
            frobA = frobA + sla.norm(a) ** 2
            count = count + 1
        else:
            # print(B)
            tau = np.minimum(tau_old, ApproximateRidgeScores(B, C, frobA, k))
            tau_D = ApproximateRidgeScores(B, D, frobA, k)
            # print(tau)
            # print(tau_D)
            for j in range(t):
                if not np.all(C[:, j] == 0):
                    prob_rej = 1.0 - tau[j] / tau_old[j]
                    roll = rng.random()
                    if roll < prob_rej:
                        C[:, j] = 0
                        tau_old[j] = 1.0
                    else:
                        tau_old[j] = tau[j]
                if np.all(C[:, j] == 0):
                    for l in range(t):
                        prob = (
                            tau_D[l]
                            * c
                            * (k * np.log(k) + k * np.log(1.0 / delta) / epsilon)
                            / t
                        )
                        roll = rng.random()
                        if roll < prob:
                            C[:, j] = D[:, l]
                            tau_old[j] = tau_D[l]
            count = 0

    idx = np.argwhere(np.all(C[..., :] == 0, axis=0))
    C = np.delete(C, idx, axis=1)

    print(np.shape(C))

    # Q = sla.lstsq(Omg @ C, OmgA)[0]
    Q = sla.lstsq(C, A)[0]
    return C, Q


def rID_streaming_ridge_leverage3(
    A, k, t, epsilon, delta, c, os, list_t=None, rng=default_rng()
):
    """
    from http://arxiv.org/abs/1511.07263
    A should be n by d
    k is the rank of the projection with theoretical guarantees.
    t is the stored column size
    epsilon: accuracy parameter
    delta: (1-delta) is the success probability
    c: oversampling parameter
    os: oversampling number for random projection
    choose epsilon and delta to be less than one.
    """
    n, d = np.shape(A)
    ll = k + os
    Omg = rng.standard_normal(size=(ll, n))

    count = 0
    nBlock = 0
    C = np.zeros((n, t))  # Stores actual column subset
    D = np.zeros((n, t))  # Stores a queue of new columns
    IC = -1 * np.ones((t)).astype(int)
    ID = np.zeros((t)).astype(int)
    frobA = 0

    Q = np.zeros((t, d))
    C_old = C
    OmgA = np.zeros((ll, d))
    OmgAOmgAt = np.zeros((ll, ll))

    tau_old = np.ones((t))  # Initialize sampling probabilities
    tau = np.zeros((t))
    probabilities = np.zeros((t))

    for i in range(d):
        a = A[:, i].T
        OmgA[:, i] = Omg @ A[:, i]
        OmgAOmgAt = OmgAOmgAt + OmgA[:, i] @ OmgA[:, i].T
        # OmgAOmgAt = scipy.linalg.blas.dsyr(1.0,OmgA[:, i].flatten(),  a=OmgAOmgAt, overwrite_a = 1) # not good because only overwrites upper triangular portion, so needs postprocessing
        #OmgAOmgAt = scipy.linalg.blas.dger(1.0,OmgA[:, i].flatten(),OmgA[:, i].flatten(), overwrite_x = 0, overwrite_y = 0, a=OmgAOmgAt, overwrite_a = 1) # not good because only overwrites upper triangular portion, so needs postprocessing
        if count < t:
            D[:, count] = A[:, i]
            ID[count] = i
            frobA = frobA + sla.norm(a) ** 2
            count = count + 1
        else:
            # tau = np.minimum(tau_old, ApproximateRidgeScores2(OmgA[:,:i], Omg, C, frobA, k))
            # tau_D = ApproximateRidgeScores2(OmgA[:,:i], Omg, D, frobA, k)
            tau, U, eig = ApproximateRidgeScores4(OmgAOmgAt, Omg, C, frobA, k) # returns eigenvalues/vectors
            tau = np.minimum( tau_old, tau )
            tau_D = ApproximateRidgeScores4(OmgAOmgAt, Omg, D, frobA, k, U=U, eig=eig)[0]
            for j in range(t):
                if IC[j] != -1:
                    prob_rej = 1.0 - tau[j] / tau_old[j]
                    roll = rng.random()
                    if roll < prob_rej:
                        C[:, j] = 0
                        tau_old[j] = 1.0
                        IC[j] = -1
                    else:
                        tau_old[j] = tau[j]

            num_sample = np.sum(IC < 0)
            # print(f'Sum tauD: {np.sum(tau)}')
            # print(f'Sum tauD: {np.sum(tau_D)}')
            # print(f'Sum: {np.sum(probabilities)}')
            idx_sample = np.random.choice(
                ID, num_sample, p=tau_D / np.sum(tau_D), replace=False
            )
            # print(num_sample)
            # print(IC)
            # print(ID)
            # print(idx_sample)
            count_sample = 0
            for j in range(t):
                if IC[j] < 0:
                    IC[j] = idx_sample[count_sample]
                    idx_D = np.where(ID == IC[j])[0][0]
                    # print(idx_D)
                    tau_old[j] = tau_D[idx_D]
                    C[:, j] = D[:, idx_D]
                    count_sample = count_sample + 1

                # if IC[j] == -1:
                #     for l in range(t):
                #         prob = tau_D[l]*c*(k*np.log(k)+k*np.log(1./delta)/epsilon)/t
                #         # prob = tau_D[l]/32.
                #         roll = rng.random()
                #         if roll < prob:
                #             C[:,j] = D[:, l]
                #             tau_old[j] = tau_D[l]
                #             IC[j] = ID[l]

            # Q[:,(nBlock)*t:(nBlock+1)*t] = sla.lstsq(Omg @ C, OmgA[:, (nBlock)*t:(nBlock+1)*t])[0]
            # if nBlock == 0:
            #     C_old = C
            # else:
            #     P = sla.lstsq(C, C_old)[0]
            #     C_old = C
            #     Q[:,:(nBlock)*t] = P @ Q[:,:(nBlock)*t]
            count = 0
            # nBlock = nBlock + 1

    # if nBlock * t < d:
    #     Q[:, (nBlock)*t:] = sla.lstsq(Omg @ C, OmgA[:, (nBlock)*t:])[0]
    C = np.unique(C, axis=1)
    # print(C)

    Q = sla.lstsq(Omg @ C, OmgA)[0]


    IC = np.unique(IC)

    return C, Q, IC


def truncateSVD_efficient(A, k):
    Q, R = sla.qr(
        A.T, mode="economic", pivoting=False, check_finite=False
    )  # A.T = QR so R.T Q.T = A
    U, s, Vh = sla.svd(R.T)  # a little m x m SVD
    Vh = Vh @ Q.T

    U_k = U[:, range(k)]
    Vh_k = Vh[range(k), :]
    s_k = s[range(k)]

    return U_k, s_k, Vh_k


def ApproximateRidgeScores_old(B, M, frobA, k):
    n, t = np.shape(M)
    tau = np.zeros((t))

    # Uk_B, Sk_B ,Vk_B = truncateSVD_efficient(B, k)
    # Bk = Uk_B @ (Sk_B.reshape((-1, 1)) * Vk_B)

    # kernel_inv = sla.pinv(B@B.T+ (frobA - sla.norm(Bk, 'fro')**2)/k * np.eye(n) )

    U, s, _ = sla.svd(B, full_matrices=False)
    eig = s**2

    BkF2 = np.sum(eig)
    ridge_kernel = U.dot(np.diag(1 / (eig + (frobA - BkF2) / k))).dot(
        U.T
    )  #! ridge leverage

    for i in range(t):
        m = M[:, i]
        tau[i] = 4.0 * m.T.dot(ridge_kernel).dot(m)

    return tau


def ApproximateRidgeScores(B, M, frobA, k):
    """
    Use frequent direction sketch to compute ridge leverage scores
    """

    n, t = np.shape(M)
    tau = np.zeros((t))

    # Uk_B, Sk_B ,Vk_B = truncateSVD_efficient(B, k)
    # Bk = Uk_B @ (Sk_B.reshape((-1, 1)) * Vk_B)

    # kernel_inv = sla.pinv(B@B.T+ (frobA - sla.norm(Bk, 'fro')**2)/k * np.eye(n) )

    U, s, _ = sla.svd(B, full_matrices=False)
    UtM = U.T @ M
    eig = s**2

    BkF2 = np.sum(eig[1:k])
    # ridge_kernel = U.dot( np.diag(1/(eig + (frobA - BkF2)/k))).dot(U.T) #! ridge leverage
    kernel = np.diag(1 / (eig + (frobA - BkF2) / k))

    for i in range(t):
        m = UtM[:, i]
        tau[i] = 4.0 * m.T.dot(kernel).dot(m)

    return tau


def ApproximateRidgeScores2(OmgA, Omg, M, frobA, k):
    """
    Use random projected sketch to compute ridge leverage scores
    """
    n, t = np.shape(M)
    ll, d = np.shape(OmgA)
    tau = np.zeros((t))

    if ll >= d:
        U, s, _ = sla.svd(OmgA, full_matrices=False)
        UtOmgM = U.T @ Omg @ M
        eig = s**2
    else:
        # _, R = sla.qr( OmgA.T, mode='economic', pivoting=False, check_finite=False ) # A.T = QR so R.T Q.T = A
        U, eig, _ = sla.svd(OmgA @ OmgA.T, full_matrices=False)  # a little m x m SVD
        UtOmgM = U.T @ Omg @ M

    BkF2 = np.sum(eig[1:k])
    kernel = np.diag(1 / (eig + (frobA - BkF2) / k))

    for i in range(t):
        m = UtOmgM[:, i]
        tau[i] = 4.0 * m.T.dot(kernel).dot(m)

    return tau


def ApproximateRidgeScores3(OmgAOmgAT, Omg, M, frobA, k):
    """
    Use random projected sketch to compute ridge leverage scores, better implementation to handle larger time steps
    """
    _, t = np.shape(M)
    tau = np.zeros((t))

    U, eig, _ = sla.svd(OmgAOmgAT, full_matrices=False)  # a little m x m SVD
    UtOmgM = U.T @ Omg @ M

    BkF2 = np.sum(eig[1:k])
    kernel = np.diag(1 / (eig + (frobA - BkF2) / k))

    for i in range(t):
        m = UtOmgM[:, i]
        tau[i] = 4.0 * m.T.dot(kernel).dot(m)

    return tau

def ApproximateRidgeScores4(OmgAOmgAT, Omg, M, frobA, k, U=None, eig=None):
    """
    Use random projected sketch to compute ridge leverage scores, better implementation to handle larger time steps
    """
    _, t = np.shape(M)
    tau = np.zeros((t))

    if (U is None) or (eig is None):
        U, eig, _ = sla.svd(OmgAOmgAT, full_matrices=False)  # a little m x m SVD

    #UtOmgM = U.T @ Omg @ M
    UtOmgM = np.linalg.multi_dot( [U.T, Omg, M] )  # selects best order of parenthesis, depending on the sizes

    BkF2 = np.sum(eig[1:k])
    kernel = np.diag(1 / (eig + (frobA - BkF2) / k))

    for i in range(t):
        m = UtOmgM[:, i]
        tau[i] = 4.0 * m.T.dot(kernel).dot(m)

    return tau, U, eig

In [3]:
method = 1
idx_data = 1

# A = np.load(
#     '/home/angranl/Documents/dataset/nstx_data_ornl_demo_50000.npy'
# ).astype("float64")
A = np.load(
    '/Users/srbecker/NoSync/ASCR/nstx_data_ornl_demo_50000.npy'
).astype("float64")
print("Matrix is loaded")

list_rank = [100, 200, 400, 800]
list_t = None

flg_random = True

rng = default_rng(1)  # !For debugging
xi = 0.005

flg_debug = False

m, n = np.shape(A)
if flg_debug:
    flg_random = False
    l = 400
    Omg = rng.standard_normal(size=(l, m))

    A = Omg @ A
    print(f"Matrix is {l} x {n}")
else:
    print(f"Matrix is {m} x {n}")

# list_rank = [10]
nReps = 1
nAlgo = 1
err = 0.0
errors = np.zeros((nReps, nAlgo))

dimReduced = 100

Matrix is loaded
Matrix is 5120 x 50000


In [ ]:

print('starting rID...')
t_start = time.time()

C, P, IC = rID_streaming_ridge_leverage3(
    A,
    dimReduced,
    dimReduced,
    0.05,
    0.1,
    1.0,
    400,
    list_t=list_t,
    rng=default_rng(),
)
# print(IC)

_, nc = np.shape(C)
print(f"Number of selected columns is {nc}")

t_end = time.time() - t_start
A_recon = C @ P
A_err = A - A_recon
err = sla.norm(A_err, "fro") / sla.norm(A, "fro")
print(
    f"Online randomized ID (k={dimReduced}, overall), relative error:\t{err:.4e}, Time: {t_end:.4f} sec"
)


#! Randomized SVD Yu
print('starting rSVD...')
t_start = time.time()
U, S, V = rSVDsp_streaming(A.T, b=dimReduced, k=dimReduced, rng=rng)
V = S.reshape((-1, 1)) * V
t_end3 = time.time() - t_start
nrmA = sla.norm(A,'fro')
norm_f = fastFrobeniusNorm(U, V, A.T, nrmA) / nrmA
print(
    "rSVDsp_unblock_streaming, relative error:\t\t\t{0:.2e}, Time: {1:.4f} sec".format(
        norm_f, t_end3
    )
)

```
Matrix is loaded
Matrix is 5120 x 50000
starting rID...
Number of selected columns is 100
Online randomized ID (k=100, overall), relative error:	1.3826e-01, Time: 151.6322 sec
starting rSVD...
rSVDsp_unblock_streaming, relative error:			1.21e-01, Time: 673.8001 sec
```

In [53]:
# AA = A[:,:10000] # A is 5120 x 50000.  With just 10,000 columns, and dim=100 for Omega, rSVD now takes 3 seconds
# AA = A[:,:20000] # now 6 seconds
# AA = A[:,:35000] # now 10 seconds
AA = A  # now 14 seconds instead of 673 seconds!!!

In [21]:
AA.flags['F_CONTIGUOUS']  # why???

AAt.flags['F_CONTIGUOUS'] # False
AA.T.flags['F_CONTIGUOUS'] # False

# "Calling ascontiguousarray makes a C-contiguous copy" ... or b = np.copy(a.transpose(0,2,1), order='C') 
#  C contiguous means row-major order;  F (Fortran) contiguous means column-major order.

False

In [5]:
%load_ext line_profiler

In [14]:
%autoreload 2
from rSVDsp import *

In [54]:
#! Randomized SVD Yu
print('starting rSVD...')
t_start = time.time()
# AAt = np.ascontiguousarray( AA.T.copy() ) # put in C order. Doesn't help, it already is!!
t_end0 = time.time() - t_start
print(f'Took {t_end0:.5f}s to transpose and copy data')
# AAt = np.copy( AA.T, order='F') # force F order
AAt = AA.T  # a view
t_start = time.time()
# %prun U, S, V = rSVDsp_streaming(AAt, b=dimReduced, k=dimReduced, rng=rng)
# %lprun -f rSVDsp_streaming U, S, V = rSVDsp_streaming(AAt, b=dimReduced, k=dimReduced, rng=rng, blocksize_A = 1)
# %lprun -f onepass_update U, S, V = rSVDsp_streaming(AAt, b=dimReduced, k=dimReduced, rng=rng, blocksize_A = 1)
U, S, V = rSVDsp_streaming(AAt, b=dimReduced, k=dimReduced, rng=rng, blocksize_A = 1)
t_end3 = time.time() - t_start
V = S.reshape((-1, 1)) * V
t_end3 = time.time() - t_start
nrmA = sla.norm(A,'fro')
norm_f = fastFrobeniusNorm(U, V, AA.T, nrmA) / nrmA
print(    "rSVDsp_unblock_streaming, relative error:\t\t\t{0:.2e}, Time: {1:.4f} sec".format( norm_f, t_end3    ))

starting rSVD...
Took 0.00004s to transpose and copy data
rSVDsp_unblock_streaming, relative error:			1.21e-01, Time: 14.8845 sec


Timer unit: 1e-06 s

Total time: 14.1003 s
File: /Users/srbecker/Repos/ASCR_DataReduction/src/rSVDsp.py
Function: onepass_update at line 103

Line #      Hits         Time  Per Hit   % Time  Line Contents
   103                                           def onepass_update(A,Omega,blockSize=1):
   104                                               """ Computes G = A@Omega and H = A.T@G in one-pass,
   105                                               loading in at most 'blockSize' rows of A at a time """
   106         1         23.0     23.0      0.0      m, n = np.shape(A)
   107         1          6.0      6.0      0.0      k = np.shape(Omega)[1]
   108                                           
   109         1          5.0      5.0      0.0      G = np.zeros((0,k))
   110         1        219.0    219.0      0.0      H = np.zeros((n,k))
   111                                               #print(f"Row-major order (C order) is: {A.flags['C_CONTIGUOUS']}")
   112                      

In [29]:
np.show_config()

blas_mkl_info:
    libraries = ['mkl_rt', 'pthread']
    library_dirs = ['/Users/srbecker/opt/anaconda3/lib']
    define_macros = [('SCIPY_MKL_H', None), ('HAVE_CBLAS', None)]
    include_dirs = ['/Users/srbecker/opt/anaconda3/include']
blas_opt_info:
    libraries = ['mkl_rt', 'pthread']
    library_dirs = ['/Users/srbecker/opt/anaconda3/lib']
    define_macros = [('SCIPY_MKL_H', None), ('HAVE_CBLAS', None)]
    include_dirs = ['/Users/srbecker/opt/anaconda3/include']
lapack_mkl_info:
    libraries = ['mkl_rt', 'pthread']
    library_dirs = ['/Users/srbecker/opt/anaconda3/lib']
    define_macros = [('SCIPY_MKL_H', None), ('HAVE_CBLAS', None)]
    include_dirs = ['/Users/srbecker/opt/anaconda3/include']
lapack_opt_info:
    libraries = ['mkl_rt', 'pthread']
    library_dirs = ['/Users/srbecker/opt/anaconda3/lib']
    define_macros = [('SCIPY_MKL_H', None), ('HAVE_CBLAS', None)]
    include_dirs = ['/Users/srbecker/opt/anaconda3/include']
Supported SIMD extensions in this NumPy install:

## Now profile the rID code

In [76]:
AA = A[:,:5000] # A is 5120 x 50000.  With just 10,000 columns, and dim=100 for Omega, rSVD now takes 3 seconds
# AA = A[:,:20000] # now 6 seconds
# AA = A[:,:35000] # now 10 seconds
# AA = A  # now 14 seconds instead of 673 seconds!!!
AA.flags['F_CONTIGUOUS'] 

True

In [91]:
# %lprun -f ApproximateRidgeScores4  C, P, IC = rID_streaming_ridge_leverage3(AA, dimReduced, dimReduced, 0.05, 0.1, 1.0, 400, list_t=list_t, rng=default_rng(), )

%lprun -f rID_streaming_ridge_leverage3  C, P, IC = rID_streaming_ridge_leverage3(AA, dimReduced, dimReduced, 0.05, 0.1, 1.0, 400, list_t=list_t, rng=default_rng(), )

A_recon = C @ P
A_err = AA - A_recon
err = sla.norm(A_err, "fro") / sla.norm(AA, "fro")
t_end=0.
print(f"Online randomized ID (k={dimReduced}, overall), relative error:\t{err:.4e}, Time: {t_end:.4f} sec")
# relative error:	1.0735e-01, or 1.2738e-01, 

flags:
False
False
Online randomized ID (k=100, overall), relative error:	1.2982e-01, Time: 0.0000 sec


Timer unit: 1e-06 s

Total time: 18.2882 s
File: /var/folders/m6/qx5hnd757jx4frp_fw3cdwkw0000gn/T/ipykernel_30416/802246270.py
Function: rID_streaming_ridge_leverage3 at line 146

Line #      Hits         Time  Per Hit   % Time  Line Contents
   146                                           def rID_streaming_ridge_leverage3(
   147                                               A, k, t, epsilon, delta, c, os, list_t=None, rng=default_rng()
   148                                           ):
   149                                               """
   150                                               from http://arxiv.org/abs/1511.07263
   151                                               A should be n by d
   152                                               k is the rank of the projection with theoretical guarantees.
   153                                               t is the stored column size
   154                                               epsilon: accuracy parameter
   155    